In [ ]:
import pandas as pd     
import numpy as np 
from tabulate import tabulate
import requests


In [ ]:
master = pd.read_parquet("./processed_data/4_timeserie_master.parquet")
print(tabulate(master, headers="keys", tablefmt="psql"))


In [ ]:
master_columns = [
    "id_cliente",
    "id_cliente_enum",
    "desc_cliente",
    "id_cerveza",
    "litros_dados_cliente",
    "litros_pedidos_cliente",
    "fecha_pedido_discreta",
    "fecha_pedido",
    "fecha_descarga",
    "fecha_descarga_discreta",
    "fecha_caducidad",
    "fecha_caducidad_discreta",
    "fecha_ultimo_refill_1_discreta",
    "fecha_ultimo_refill_2_discreta",
    "fecha_ultimo_refill_3_discreta",  
]

web_columns = [
    "desc_cerveza",
    "desc_cliente",
    "id_cliente",
    "fecha_descarga",
    "fecha_pedido",
    "litros_dados_cliente",
    "litros_pedidos_cliente",
    "pedido_festivo"
]

# Corrected code to subset columns
master_data = master[master_columns]
web_data = master[web_columns]

In [ ]:
#Store in a json and parquet
master_data.to_parquet("./processed_data/5_historical_master.parquet", index=True)

## Construir la BAR INFO i guardar el JSON per la WEB

Si tinguessim caleres fariem servir la API de google Maps 

In [ ]:
"""
def get_google_maps_address(place_name):
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={}".format(
        place_name
    )
    headers = {
        "Authorization": "YOUR_API_KEY",
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    place = data["candidates"][0]
    address = place["formatted_address"]
    return address


# Example

place_name = "HOTEL REINA SOFIA"
address = get_google_maps_address(place_name)

# Real data frame operation to be performed 
adresses = web_data[["desc_cliente", "id_cliente"]].drop_duplicates().apply(lambda row: get_google_maps_adress(row["desc_cliente"]), axis=1)
#ciudad remaining, should be somewhere in the string, needs to be parsed using regular expressions probably
web_data = pd.merge(adresses[["id_cliente", "direccion"]], web_data, on=["id_cliente"])
"""


In [ ]:

#Cal reordenar la estructura del dataframe, si agrupem pels bars (4 columnes) i convertim els demes features a llistes 
web_grouped = web_data.groupby(['id_cliente']).agg({
    'fecha_descarga': list,
    'fecha_pedido':list,
    'pedido_festivo': list,
    'litros_dados_cliente': list,
    'litros_pedidos_cliente': list,
    'desc_cerveza': list,
}).reset_index()

city_prob = {
    "Barcelona": 0.5,
    "Hospitalet de Llobregat": 0.09,
    "Cornella de Llobregat": 0.05,
    "Badalona": 0.10,
    "Castelldefels": 0.03,
    "Sabadell": 0.01,
    "Sitges": 0.04,
    "Tarragona": 0.03,
    "Salou": 0.03,
    "Girona": 0.07,
    "Madrid": 0.03,
    "Sevilla": 0.02
}

web_grouped["direccion"] = "C/ Jordi Girona, 1-3 Edifici Omega, Despatx 002-102, 08034 Barcelona"
web_grouped["ciudad"] = np.random.choice(list(city_prob.keys()), size=len(web_grouped), p=list(city_prob.values()))

#Add some example info
web_grouped.loc[web_grouped["id_cliente"] == 104530, ["direccion", "ciudad"]] = ["Passatge de la Banca, 7, Ciutat Vella, 08002 Barcelona", "Barcelona"]
web_grouped.loc[web_grouped["id_cliente"] == 123507, ["direccion", "ciudad"]] = ["C/ Nou de la Rambla, 113, Sants-Montjuïc, 08004 Barcelona", "Barcelona"]
web_grouped.loc[web_grouped["id_cliente"] == 125901, ["direccion", "ciudad"]] = ["C/ dels Almogàvers, 122, Sant Martí, 08018 Barcelona", "Barcelona"]
web_grouped.loc[web_grouped["id_cliente"] == 104670, ["direccion", "ciudad"]] = ["Pl. de Catalunya, 21, Ciutat Vella, 08002 Barcelona", "Barcelona"]
web_grouped.loc[web_grouped["id_cliente"] == 104749, ["direccion", "ciudad"]] = ["Pl. Pius XII, 4, Les Corts, 08028 Barcelona, 08002 Barcelona", "Barcelona"]
web_grouped.loc[web_grouped["id_cliente"] == 119680, ["direccion", "ciudad"]] = ["Via Laietana, 40, Ciutat Vella, 08003 Barcelona", "Barcelona"]
web_grouped.loc[web_grouped["id_cliente"] == 104804, ["direccion", "ciudad"]] = ["Carrer de Fuerteventura, s/n, 17210 Calella de Palafrugell, Girona", "Calella de Palafrugell"]


#Get the client info
web_grouped = pd.merge(web_data[["id_cliente", "desc_cliente"]].drop_duplicates(), web_grouped, on=["id_cliente"])
web_grouped


In [ ]:
#Sanity check, size of all grouped features
print(len(web_grouped.iloc[0]["fecha_pedido"]))
print(len(web_grouped.iloc[0]["desc_cerveza"]))
print(len(web_grouped.iloc[0]["pedido_festivo"]))
print(len(web_grouped.iloc[0]["fecha_descarga"]))
print(len(web_grouped.iloc[0]["litros_dados_cliente"]))
print(len(web_grouped.iloc[0]["litros_pedidos_cliente"]))

In [ ]:
"""
submaster = master[["id_cliente", "desc_cliente", "fecha_pedido_discreta"]].sort_values(by=["id_cliente", "fecha_pedido_discreta"])
grouped_master = submaster.groupby("id_cliente")
arrays_by_id_cliente = submaster.groupby("id_cliente")["fecha_pedido_discreta"].apply(lambda x: x.to_numpy()).to_dict()
print(arrays_by_id_cliente[104530])
"""

In [ ]:
web_grouped.to_json("./processed_data/5_web_ready_data.json", orient="records", indent=2, date_format="iso")